# Going Subphraseless

The current method for isolating phrase heads ([here](https://nbviewer.jupyter.org/github/ETCBC/heads/blob/master/phrase_heads.ipynb)) requires strenuous and ineloquent processing of BHSA subphrase relations. The subphrases are not always consistently encoded and suffer from numerous exceptional cases. The result is that the method is rather convoluted and ineloquent.

This notebook will explore the possibility of disconnecting semantic head analysis from the ETCBC subphrase encoding. 

A "semantic" head is the primary content word of a phrase, following Croft's "Primary Information Bearing Unit":

> **The noun and the verb are the PRIMARY INFORMATION_BEARING UNITS (PIBUs) of the phrase and clause respectively. In common parlance, they are the content words. PIBUs have major informational content that functional elements such as articles and [auxiliaries] do not have. (Croft, *Radical Construction Grammar*, 2001, 258; see also Shead, *Radical Frame Semantics and Biblical Hebrew*, 104)**

> **A (semantic) head is the profile equivalent that is the primary information-bearing unit, that is, the most contentful item that most closely profiles the same kind of thing that the whole constituent profiles. (ibid., 259)**

Croft also provides an additional criterion to "profile equivalence":

> **If the criterion of profile equivalence produces two candidates for headhood, the less schematic meaning is the PIBU; that is, the PIBU is the one with the narrower extension, in the formal semantic sense of that term (ibid., 259)**

## Inquiry

Can we isolate semantic phrase heads in BHSA using only the phrase_atom and phrase limits? This question indeed means that we  take the phrase_atom/phrase boundaries for granted. Empirically, the validity of BHSA phrase boundaries needs to be tested. But for now, the exercise of isolating semantic phrase heads could be seen as the first step towards reproducible phrase boundaries.

## Basic Concepts

A semantic head will most often stand in a syntactically independent position. For Hebrew nominal phrases, that essentially means a word which is not precided by a construct, which is not in an attributive slot (e.g. H + noun + H + ATTRIBUTIVE), and which is not in an adjectival slot (e.g. noun + noun as in אישׁ טוב).

The situation is slightly complicated by quantifier expressions, which may be syntactically independent but semantically secondary. These are expressed through specialized lexical items such as cardinal numbers and qualitative quantifiers (e.g.  "כל" and "חצי").

Another complication is the use of nouns as prepositional items. Such uses can be seen with words like פני "face" such as לפני "in front," and even words like ראשׁ as in ראשׁ החדשׁ "beginning of the month." 

Other expressions of quantity, quality, and function provide similar complexities. These cases have to be specified in advance.

## Prerequisites

As discussed above, lexical-semantic information is crucial in separating instances of quantification. These sets have to be defined in advance. As already noted, the BHSA phrase_atom/phrase boundaries are also taken for granted.

## The Noun Phrase

The focus of this initial inquiry is the noun phrase. All of the most complicated problems can be found in this group. Solving the problems in NP classification will likewise allow PP classification to easily follow. In cognitive linguistics, nouns are considered the semantic heads of prepositional phrases. That same definition will be adopted herein.

### A Process of Elimination

This exploratory analysis will proceed via a process of elimination, advancing from simple cases towards the complex ones.

**Let's get started**. We load the necessary functions and BHSA data (straight from source).

In [1]:
import collections
import random
from tf.app import use
A = use('bhsa', hoist=globals())

	connecting to online GitHub repo annotation/app-bhsa ... connected
Using TF-app in /Users/cody/text-fabric-data/annotation/app-bhsa/code:
	rv1.2=#5fdf1778d51d938bfe80b37b415e36618e50190c (latest release)
	connecting to online GitHub repo etcbc/bhsa ... connected
Using data in /Users/cody/text-fabric-data/etcbc/bhsa/tf/c:
	rv1.6 (latest release)
	connecting to online GitHub repo etcbc/phono ... connected
Using data in /Users/cody/text-fabric-data/etcbc/phono/tf/c:
	r1.2 (latest release)
	connecting to online GitHub repo etcbc/parallels ... connected
Using data in /Users/cody/text-fabric-data/etcbc/parallels/tf/c:
	r1.2 (latest release)
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used


As a first step, we need to define what word types will become elligible candidates for noun heads. Parts of speech are frequently construed into various roles. As a result, we must be prepared to accept a variety of potential candidates.

Instead of the BHSA `pdp` ("phrase dependent part of speech") feature, we will rely on the `sp` ("part of speech") feature, which is derived directly from the lexicon.

Let's have a look at the potential `sp` values in the dataset.

In [2]:
F.sp.freqList()

(('subs', 125581),
 ('verb', 75450),
 ('prep', 73298),
 ('conj', 62737),
 ('nmpr', 35696),
 ('art', 30387),
 ('adjv', 10052),
 ('nega', 6059),
 ('prps', 5035),
 ('advb', 4603),
 ('prde', 2678),
 ('intj', 1912),
 ('inrg', 1303),
 ('prin', 1026))

See the short descriptions for the values [here](https://etcbc.github.io/bhsa/features/sp/). 

The following parts of speech could be expected to be construed as NP sem. heads: substantives (`subs`), verbs (i.e. as participles), proper nouns (`nmpr`), adjectives (`adjv`), personal pronouns (`prps`, most often found exclusively in phrases marked a "Personal Pronoun Phrases", but in cases of noun coordination BHSA will not distinguish!), demonstrative pronouns (`prde`). 

It is unknown whether adverbs (`advb`) or interrogative particles (`inrg`) might also function as semantic heads in BHSA phrases. Let's do a quick check. We look for cases where a word with a `sp` value of `advb` becomes a `subs` in its `pdp` feature.

In [3]:
find_advb = A.search('word sp=advb pdp=subs')

  0.54s 0 results


We don't find any cases. How about the interrogative particle inside phrases marked as `NP`?

In [9]:
find_intr = A.search('''

clause
    phrase typ=NP
        word sp=inrg

''')

  0.72s 1 result


In [10]:
A.table(find_intr, condenseType='clause')

n,p,clause,phrase,word
1,1_Chronicles 17:6,בְּכֹ֥ל הֲדָבָ֣ר דִּבַּ֗רְתִּי אֶת־אַחַד֙ שֹׁפְטֵ֣י יִשְׂרָאֵ֔ל,הֲדָבָ֣ר,הֲ


The interrogative does not function as a semantic head here.

Ok. So this leaves us with the other potential part of speech values. Are these iron-clad? Let's do this: we can be even more sure we have the right set by making a broader query: for any word with a `pdp=subs` wherein the `sp != subs`.

We expect to see only: `subs`, `nmpr`, and `adjv`. The values `prps` and `prde` are not likely to change in `pdp`.

In [15]:
set(F.sp.v(w) for w in F.otype.s('word')
        if F.pdp.v(w) == 'subs' and F.sp.v(w) != 'subs')

{'adjv', 'verb'}

This is *mostly* what we expected. But I did forget that `nmpr` will also *not* change inside the NP. So `adjv` and `verb` is exactly what we want to see here.

I see that I've also included `prde` "demonstratives" in the list here. I now very much doubt whether this is ever the case. I'll make a query here to see whether any demonstratives occur outside of a modifying position. Specifically, we require that the demonstrative is not preceded by an article (excludes the attributive pattern H + noun + H + demonstrative) or a noun, and the demonstrative does not precede a noun. 

In [27]:
find_demo = A.search('''

phrase typ=NP
    word sp#art|subs|adjv
    <: word sp=prde
    <: word sp#subs|adjv
''')

  1.80s 2 results


In [29]:
A.table(find_demo, withNodes=True)

n,p,phrase,word,word,word
1,Exodus 26:13,הָאַמָּ֨ה מִזֶּ֜ה וְהָאַמָּ֤ה מִזֶּה֙ 678044,מִ 42881,זֶּ֜ה 42882,וְ 42883
2,2_Chronicles 9:18,יָדֹ֛ות מִזֶּ֥ה וּמִזֶּ֖ה 896933,מִ 411458,זֶּ֥ה 411459,וּ 411460


These cases are slightly complicated by the fact that at a higher level they do govern their own phrase, but at the level of the whole phrase they do not. Is this fact reflected in the BHSA phrase structure?

In [32]:
A.pretty(678044, withNodes=True)

So we see a separate phrase atom encoded here. What kind of phrase type value is coded on that phrase atom?

In [35]:
F.typ.v(L.u(42882, 'phrase_atom')[0])

'PP'

Ok, BHSA simply encodes this as a prepositional phrase. However, our algorithm ought to work also with such prepositional phrases. This presents a slight complicating factor. On the one hand, a demonstrative pronoun might be treated like any other substantive, yet it is functionally different in nearly all cases. For example, many cases would consist of a demonstrative + noun pattern. The algorithm might see: noun_candidateA (זה) + noun_candidateB; it sees that noun_candidateB agrees with noun_candidateA in gender and number; pursuant to the rule on adjectives, i.e. Gesenius §132.1a, the algorithm (incorrectly) classifies noun_candidateA as syntactically autonomous and thus the top candidate for semantic headedness.

What to do?

The idea of a "ranking" system is intriguing. In such a system, cases like this receive a ranking based purely on the syntactic patterning. But we can introduce a lexical processing stage that adjusts ranks based on lexical rules. That also helps us handle complex cases such as quantifiers. 

Other cases may be very complicated. An example can be found in the following English noun phrase:

> a. Tim drank a **cup** of coffee.<br>
> b. \*Tim broke a **cup** of coffee.

Syntactically "cup" is the expected head, but semantically the coffee is the head. This is indicated by the "cup of" construction in English, which indicates a measure of a substance. A heads algorithm needs to be able to take into account such idiosyncrasies. That information simply has to be hardwired in.

Great! Now we can confidently say that valid NP head candidates should have only the following `sp` values: `subs`, `nmpr`, `adjv`, `prps`, with some allowances made for `prde`.